### get pool statistics using coingecko API

In [1]:
# necessary imports
import requests # Importing the requests library for making HTTP requests
import pandas as pd # Importing pandas for data manipulation and analysis
import json # Importing json for handling JSON data
import os # Importing os for operating system dependent functionality
from dotenv import load_dotenv # Importing load_dotenv to load environment variables from a .env file
import joblib
import sqlite3


In [2]:
# show all rows
# pd.set_option("display.max_rows", None)

#show all columns
# pd.set_option("display.max_columns", None)

#show full column content
# pd.set_option("display.max_colwidth", None)

In [3]:
load_dotenv()

GECKO_API = os.getenv("GECKO_API")
if not GECKO_API:
    raise ValueError("GECKO_API environment variable is not set.")

In [4]:
# Define network and token
network = "eth"
token_address = "0xdAC17F958D2ee523a2206206994597C13D831ec7"  # USDT

# Format URL properly
url = f"https://pro-api.coingecko.com/api/v3/onchain/networks/{network}/tokens/{token_address}/pools"

# Get API key from environment variable
API_KEY = os.getenv("GECKO_API")
if not API_KEY:
    raise ValueError("GECKO_API environment variable not set.")

headers = {
    "accept": "application/json",
    "x-cg-pro-api-key": API_KEY
}

# Make request
response = requests.get(url, headers=headers)
response.raise_for_status()
data = response.json()

# Pretty-print response
# print(f"API Response:\n{json.dumps(data, indent=4)}")

In [5]:
# Extract attributes into a list
attributes_list = [item["attributes"] for item in data["data"]]

# Convert to DataFrame
df = pd.DataFrame(attributes_list)

df.head()

,base_token_price_usd,base_token_price_native_currency,quote_token_price_usd,quote_token_price_native_currency,base_token_price_quote_token,quote_token_price_base_token,address,name,pool_created_at,token_price_usd,fdv_usd,market_cap_usd,price_change_percentage,transactions,volume_usd,reserve_in_usd
0,0.999159555408533,0.000234545749126468,0.99950082195496,0.000234625859072222,0.999658563,1.0003415536,0x8aa4e11cbdf30eedc92100f4c8a31ff748e201d44712...,USDC / USDT 0.001%,2025-04-01T02:10:47Z,0.99950082195496,80745525731.6106,166649850953.28,"{'m5': '0.08', 'm15': '-0.02', 'm30': '0', 'h1...","{'m5': {'buys': 3, 'sells': 0, 'buyers': 3, 's...","{'m5': '15505.9990507254', 'm15': '196830.8063...",22422011.034
1,4259.9772501943,1.0,0.999866465880289,0.000234625859072222,4262.104799336,0.0002346258591,0x72331fcb696b0151904c03584b66dc8365bc63f8a144...,ETH / USDT 0.05%,2025-01-27T22:08:23Z,0.999866465880289,80775064587.7403,166649850953.28,"{'m5': '0.04', 'm15': '-0.11', 'm30': '0.06', ...","{'m5': {'buys': 8, 'sells': 5, 'buyers': 8, 's...","{'m5': '307478.249956073', 'm15': '1262717.494...",83264147.4062
2,4270.42473947516,1.0,1.00314940021038,0.000234625859072222,4262.104799336,0.0002346258591,0x4e68ccd3e89f51c3074ca5072bbac773960dfa36,WETH / USDT 0.3%,2021-12-29T12:36:12Z,1.00314940021038,81040279235.4952,166649850953.28,"{'m5': '0', 'm15': '0', 'm30': '0.01', 'h1': '...","{'m5': {'buys': 13, 'sells': 0, 'buyers': 13, ...","{'m5': '721115.945561588', 'm15': '1523539.721...",73589988.991
3,0.991485894653521,0.000233343597911684,0.996579742499797,0.000234073471406423,0.9968818615,1.0031278917,0xbebc44782c7db0a1a60cb6fe97d0b483032ff1c7,DAI / USDC / USDT,2022-07-13T11:48:12Z,0.991485894653521,4139078894.81834,4128352088.51572,"{'m5': '0', 'm15': '0', 'm30': '0', 'h1': '-0....","{'m5': {'buys': 0, 'sells': 0, 'buyers': None,...","{'m5': '0.0', 'm15': '199.289910861', 'm30': '...",194277440.3164
4,0.999054888786143,0.000232487029794902,0.999175562728141,0.000232515111461549,0.9998792265,1.0001207881,0xd9f673912e1da331c9e56c5f0dbc7273c0eb68461793...,USDC / USDT,2025-07-22T18:57:11Z,0.999175562728141,80719249387.7681,166649850953.28,"{'m5': '0', 'm15': '0', 'm30': '0', 'h1': '-0....","{'m5': {'buys': 0, 'sells': 0, 'buyers': 0, 's...","{'m5': '0.0', 'm15': '4939570.80294633', 'm30'...",26928629.6368


In [6]:
os.makedirs("Database", exist_ok=True)
file_path = "/home/realist/projects/DexTracker/backend/Database/usdt_pairs.joblib"
joblib.dump(df, file_path)


['/home/realist/projects/DexTracker/backend/Database/usdt_pairs.joblib']

In [ ]:
#load saved data
df = joblib.load(file_path)

# usdc/usdt pool    address 0x8aa4e11cbdf30eedc92100f4c8a31ff748e201d44712cc8c90d189edaa8e4e47

In [ ]:
if not isinstance(df, pd.DataFrame):
    df = pd.DataFrame([df])

df = df.map(lambda x: str(x) if isinstance(x, (dict, list)) else x)



conn = sqlite3.connect("/home/realist/projects/DexTracker/backend/Database/usdt_pool_pairs.db")

df.to_sql("usdt_pool_pairs", conn, if_exists="replace", index=False)

result = pd.read_sql("SELECT * FROM usdt_pool_pairs LIMIT 5", conn)
result.head()



,base_token_price_usd,base_token_price_native_currency,quote_token_price_usd,quote_token_price_native_currency,base_token_price_quote_token,quote_token_price_base_token,address,name,pool_created_at,token_price_usd,fdv_usd,market_cap_usd,price_change_percentage,transactions,volume_usd,reserve_in_usd
0,0.999159555408533,0.000234545749126468,0.99950082195496,0.000234625859072222,0.999658563,1.0003415536,0x8aa4e11cbdf30eedc92100f4c8a31ff748e201d44712...,USDC / USDT 0.001%,2025-04-01T02:10:47Z,0.99950082195496,80745525731.6106,166649850953.28,"{'m5': '0.08', 'm15': '-0.02', 'm30': '0', 'h1...","{'m5': {'buys': 3, 'sells': 0, 'buyers': 3, 's...","{'m5': '15505.9990507254', 'm15': '196830.8063...",22422011.034
1,4259.9772501943,1.0,0.999866465880289,0.000234625859072222,4262.104799336,0.0002346258591,0x72331fcb696b0151904c03584b66dc8365bc63f8a144...,ETH / USDT 0.05%,2025-01-27T22:08:23Z,0.999866465880289,80775064587.7403,166649850953.28,"{'m5': '0.04', 'm15': '-0.11', 'm30': '0.06', ...","{'m5': {'buys': 8, 'sells': 5, 'buyers': 8, 's...","{'m5': '307478.249956073', 'm15': '1262717.494...",83264147.4062
2,4270.42473947516,1.0,1.00314940021038,0.000234625859072222,4262.104799336,0.0002346258591,0x4e68ccd3e89f51c3074ca5072bbac773960dfa36,WETH / USDT 0.3%,2021-12-29T12:36:12Z,1.00314940021038,81040279235.4952,166649850953.28,"{'m5': '0', 'm15': '0', 'm30': '0.01', 'h1': '...","{'m5': {'buys': 13, 'sells': 0, 'buyers': 13, ...","{'m5': '721115.945561588', 'm15': '1523539.721...",73589988.991
3,0.991485894653521,0.000233343597911684,0.996579742499797,0.000234073471406423,0.9968818615,1.0031278917,0xbebc44782c7db0a1a60cb6fe97d0b483032ff1c7,DAI / USDC / USDT,2022-07-13T11:48:12Z,0.991485894653521,4139078894.81834,4128352088.51572,"{'m5': '0', 'm15': '0', 'm30': '0', 'h1': '-0....","{'m5': {'buys': 0, 'sells': 0, 'buyers': None,...","{'m5': '0.0', 'm15': '199.289910861', 'm30': '...",194277440.3164
4,0.999054888786143,0.000232487029794902,0.999175562728141,0.000232515111461549,0.9998792265,1.0001207881,0xd9f673912e1da331c9e56c5f0dbc7273c0eb68461793...,USDC / USDT,2025-07-22T18:57:11Z,0.999175562728141,80719249387.7681,166649850953.28,"{'m5': '0', 'm15': '0', 'm30': '0', 'h1': '-0....","{'m5': {'buys': 0, 'sells': 0, 'buyers': 0, 's...","{'m5': '0.0', 'm15': '4939570.80294633', 'm30'...",26928629.6368


In [18]:
with sqlite3.connect("/home/realist/projects/DexTracker/backend/Database/usdt_pool_pairs.db") as conn:
    df_table = pd.read_sql("""
    SELECT
    name,
    market_cap_usd,
    reserve_in_usd
    FROM usdt_pool_pairs
    WHERE 1=1
    AND address = '0x8aa4e11cbdf30eedc92100f4c8a31ff748e201d44712cc8c90d189edaa8e4e47'
    """,conn)

df_table

,name,market_cap_usd,reserve_in_usd
0,USDC / USDT 0.001%,166649850953.28,22422011.034


In [20]:
df_table = df_table.astype({
    "name": "category",
    "market_cap_usd": "float64",
    "reserve_in_usd": "float64"
})